### Przygotowanie danych

In [ ]:
# pip install requirements.txt

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sktime.transformations.series.impute import Imputer
import plotly.express as px
import warnings

In [ ]:
df_filtr_std = pd.read_csv("../chapter1/filtered_std_data_gasometry.csv")

### Poszukiwanie najlepszej metody uzupełniania braków danych

In [ ]:
#Działanie na przefiltrowanych i ujednolicenie zmiennych 
df_imp=df_filtr_std.copy()
cols=["BETET",	"CO2TET",	"HCO3ACTE",	"HCO3STTE",	"O2SATTET",	"O2TET", "IONH", "ZGON"]
df_imp["czy_na"]=df_imp[cols].apply(lambda row: any(row.isna()), axis=1)
id=df_imp[df_imp["czy_na"]==True]["PACJENT_NR"]
df_bezna = df_imp[~df_imp['PACJENT_NR'].isin(id)]
df_bezna = df_bezna[["PACJENT_NR", *cols]]
df_bezna

In [ ]:
df_imp[df_imp["czy_na"]==True]["PACJENT_NR"]

In [ ]:
np.random.seed(13)
num_values_to_remove =int(1000)
rows_to_remove = np.random.choice(df_bezna.index, size=num_values_to_remove, replace=True)
cols_to_remove = np.random.choice(df_bezna.columns[1:10], size=num_values_to_remove, replace=True)
id=zip(rows_to_remove, cols_to_remove)
df_nowe=df_bezna.copy()

for row, col in id:
  df_nowe.loc[row, col] = np.nan

df_nowe.isnull().sum().sum()/19250

In [ ]:
#KNN na wszystkich danych
wyn_knn_all=list()
n_range=range(1,31)
for i in n_range:
  kmni_all = KNNImputer(n_neighbors=i)
  kmni=kmni_all.fit_transform(df_nowe[cols])
  wyn_knn_all.append(np.sqrt(mean_squared_error(df_bezna[cols], kmni)))

wyn_knn_all_df=pd.DataFrame(np.column_stack([n_range, wyn_knn_all]), columns=["k","rmse"])

fig = px.line(wyn_knn_all_df, x="k", y="rmse", markers=True, line_shape='linear')

# Dodajemy etykiety i tytuł
fig.update_layout(
    xaxis=dict(
        title='Liczba sąsiadów (k)',
        tickfont=dict(size=16),
        title_font=dict(size=20),
    ),
    yaxis=dict(
        title='Miara RMSE',
        tickfont=dict(size=16),
        title_font=dict(size=20)
    )
)
fig.update_layout(template="plotly_white")

# Wyświetlamy wykres
fig.show()
fig.write_image("images3/knn_all.png", width=1000, height=600, scale=4, format="png")

In [ ]:
wyn_knn_all_df.iloc[wyn_knn_all_df["rmse"].idxmin()]

In [ ]:
# Inicjalizujemy słownik do przechowywania wyników
pacjenci_df_knn = {}
n_neighbors_range = range(1, 11)
wyn_knn_pac=list()

for n_neighbors in n_neighbors_range:
    knn_imp = KNNImputer(n_neighbors=n_neighbors)
    pacjenci_df_knn = np.concatenate([knn_imp.fit_transform(pacjent_df[cols]) for _, pacjent_df in df_nowe.groupby(by='PACJENT_NR')],axis=0)
    pacjenci_df_knn= pd.DataFrame(pacjenci_df_knn, columns=cols)
    wyn_knn_pac.append(np.sqrt(mean_squared_error(df_bezna[cols], pacjenci_df_knn)))

In [ ]:
wyn_knn_pac_df=pd.DataFrame(np.column_stack([n_neighbors_range, wyn_knn_pac]), columns=["k","rmse"])

fig = px.line(wyn_knn_pac_df, x="k", y="rmse", markers=True, line_shape='linear')

# Dodajemy etykiety i tytuł
fig.update_layout(
    xaxis=dict(
        title='Liczba sąsiadów (k)',
        tickfont=dict(size=16),
        title_font=dict(size=20),
    ),
    yaxis=dict(
        title='Miara RMSE',
        tickfont=dict(size=16),
        title_font=dict(size=20)
    )
)
fig.update_layout(template="plotly_white")
# Wyświetlamy wykres
fig.show()
fig.write_image("images3/knn_patient.png", width=1000, height=600, scale=4, format="png")


In [ ]:

imputers = {
    "mean": Imputer(method="mean"),
    "median": Imputer(method="median"),
    "drift": Imputer(method="drift"),
    "backfill": Imputer(method="backfill"),
    "ffill": Imputer(method="pad"),
    "knn_p": KNNImputer(n_neighbors=2)
}

pacjenci_df = {}

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for imp_name, imp in imputers.items():
        pacjenci_df[imp_name] = np.concatenate(
            [imp.fit_transform(pacjent_df[cols]) for _, pacjent_df in df_nowe.groupby(by='PACJENT_NR')],
            axis=0
        )
        pacjenci_df[imp_name] = pd.DataFrame(pacjenci_df[imp_name], columns=cols)

kmni_imputer = KNNImputer(n_neighbors=12)
pacjenci_df["knn_all"]=kmni_imputer.fit_transform(df_nowe[cols])


In [ ]:
print("Podstawienie średniego wyniku pacjenta: ", str(np.round(mean_squared_error(df_bezna[cols], pacjenci_df['mean']), 4)))
print("Podstawienie mediany wyników pacjenta: ", str(np.round(mean_squared_error(df_bezna[cols], pacjenci_df['median']), 4)))
print("drift ", str(np.round(mean_squared_error(df_bezna[cols], pacjenci_df['drift']), 4)))
print("ffill ", str(np.round(mean_squared_error(df_bezna[cols], pacjenci_df['ffill']), 4)))
print("backfill ", str(np.round(mean_squared_error(df_bezna[cols], pacjenci_df['backfill']), 4)))
print("Metoda najbliższych sąsiadów k=2 na pacjentach: ", str(np.round(mean_squared_error(df_bezna[cols], pacjenci_df['knn_p']),4)))
print("Metoda najbliższych sąsiadów k=12 na wszystkich: ", str(np.round(mean_squared_error(df_bezna[cols], pacjenci_df["knn_all"]),4)))

### Uzupełnienie danych najlepszą metoda

In [ ]:
#wypełnianie braków metodą K najbliższych sąsiadów
imputer = KNNImputer(n_neighbors=12)
df_prepared = pd.DataFrame(imputer.fit_transform(df_filtr_std[cols]), columns=cols)
df_prepared


In [ ]:
info_cols=df_filtr_std[["PACJENT_NR", "BADANIE_NR"]]
df_prepared2=info_cols.merge(df_prepared, left_index=True, right_index=True)
df_prepared2.to_csv("gasometry_prepared_data.csv")